In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from koopmanlib.param_solver import KoopmanParametricDLSolver, KoopmanLinearDLSolver, KoopmanBilinearDLSolver
from koopmanlib.dictionary import PsiNN
from tensorflow.keras.optimizers import Adam
import json
import sys
import numpy as np
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


2024-03-06 18:32:50.843518: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
config_file = 'config_vdpm.json'

In [4]:

# config_file = sys.argv[1]
with open(config_file, 'r') as f:
    config = json.load(f)

data_path = config['data_settings']['data_path']
weights_path = config['nn_settings']['weights_path']

n_psi_train = config['nn_settings']['n_psi_train']
mu_list = config['data_settings']['mu']


target_dim = 2
param_dim = 1

n_psi = 1 + target_dim + n_psi_train

dict_layer_size = config['nn_settings']['dict_layer_size']
K_layer_size_list = config['nn_settings']['K_layer_size']

linear_epochs = config['nn_settings']['linear_epochs']
bilinear_epochs = config['nn_settings']['bilinear_epochs']
pknn_epochs = config['nn_settings']['pknn_epochs']


In [5]:
dict_layer_size

[32, 32]

In [6]:

# class CustomLoss(tf.keras.losses.Loss):
#     def __init__(self, time_step, **kwargs):
#         super().__init__(**kwargs)
#         self.time_step = time_step

#     def call(self, y_true, y_pred):  
   
#         # # mse loss on psi
#         # mse = tf.reduce_mean(tf.square(tf.norm(psi_next-psi_y, axis=-1))) / (self.time_step)**2

#         # mse loss on psi
#         self.mse = tf.square(tf.norm(y_true-y_pred, axis=-1)) / (self.time_step**2)
#         return self.mse
    
# my_loss = CustomLoss(time_step=0.01)


In [7]:

mu = mu_list[0]
K_layer_size = K_layer_size_list[0]

# Load data
dict_data = np.load(os.path.join(
    data_path, 'vdpm_data_mu_'+str(mu)+'.npy'), allow_pickle=True)

data_x = dict_data[()]['data_x']
data_y = dict_data[()]['data_y']
data_u = dict_data[()]['data_u']

# PK-NN
dic_pk = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train, name='psi_nn_layer')
from koopmanlib.K_structure import Model_K_u_Layer, Model_K_u_Layer_One

model_K_u = Model_K_u_Layer_One(layer_sizes=K_layer_size, 
                                n_psi=n_psi,
                                activation='tanh')

solver_pk = KoopmanParametricDLSolver(
    target_dim=target_dim, 
    param_dim=param_dim, 
    n_psi=n_psi, 
    dic=dic_pk, 
    model_K_u=model_K_u)

model_pk, model_K_u_pred_pk = solver_pk.generate_model()


2024-03-06 18:32:53.817839: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.3/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64
2024-03-06 18:32:53.817866: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-03-06 18:32:53.818358: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 A

In [8]:

zeros_data_y_train = tf.zeros_like(dic_pk(data_y))


In [9]:
lr = 1e-3
opt = Adam(lr)

model_pk.compile(optimizer=opt,
                    loss='mse')
    


In [10]:

lr_callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                                    factor=0.1,
                                                    patience=100,
                                                    verbose=0,
                                                    mode='auto',
                                                    min_delta=1e-4,
                                                    cooldown=0,
                                                    min_lr=1e-6)

# Define the early stopping criteria
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                  min_delta=1e-9,
                                                  patience=50, 
                                                  verbose=1, 
                                                  mode='auto')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(
        weights_path, 'model_pk_vdpm_mu_'+str(mu)+'.h5'),
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=True,
        mode='min',
        save_freq='epoch')

# Add early_stopping to the list of callbacks
callbacks = [lr_callbacks, es_callback, checkpoint_callback]


In [23]:

history = model_pk.fit(x=[data_x, data_y, data_u],
                    y=zeros_data_y_train,
                    validation_split=0.2,
                    epochs=pknn_epochs,
                    batch_size=200,
                    callbacks=callbacks,
                    verbose=1)

Epoch 1/1000
100/100 [==============================] - 0s 3ms/step - loss: 2.7259e-08 - val_loss: 2.1876e-07 - lr: 1.0000e-06
Epoch 2/1000
100/100 [==============================] - 0s 3ms/step - loss: 2.7223e-08 - val_loss: 2.1570e-07 - lr: 1.0000e-06
Epoch 3/1000
100/100 [==============================] - 0s 3ms/step - loss: 2.7222e-08 - val_loss: 2.1235e-07 - lr: 1.0000e-06
Epoch 4/1000
100/100 [==============================] - 0s 3ms/step - loss: 2.7231e-08 - val_loss: 2.1564e-07 - lr: 1.0000e-06
Epoch 5/1000
100/100 [==============================] - 0s 3ms/step - loss: 2.7192e-08 - val_loss: 2.1801e-07 - lr: 1.0000e-06
Epoch 6/1000
100/100 [==============================] - 0s 3ms/step - loss: 2.7177e-08 - val_loss: 2.1212e-07 - lr: 1.0000e-06
Epoch 7/1000
100/100 [==============================] - 0s 3ms/step - loss: 2.7207e-08 - val_loss: 2.1807e-07 - lr: 1.0000e-06
Epoch 8/1000
100/100 [==============================] - 0s 3ms/step - loss: 2.7192e-08 - val_loss: 2.2424e-07 -

In [24]:
data_x.shape

(25000, 2)

In [12]:
# model_pk.save_weights(os.path.join(
#         weights_path, 'model_pk_vdpm_mu_'+str(mu)+'.h5'))

In [13]:

# model_pk.get_layer('DicNN').trainable = False
# for layer in solver_pk.model_K_u.hidden_layers:
#     layer.trainable = False
# solver_pk.model_K_u.output_layer.trainable = True


In [14]:
# model_pk.summary()

In [15]:
# model_pk.compile(optimizer=Adam(lr=1e-4),
#                     loss='mse')


In [16]:
# history = model_pk.fit(x=[data_x, data_y, data_u],
#                     y=zeros_data_y_train,
#                     validation_split=0.2,
#                     epochs=100,
#                     batch_size=200,
#                     callbacks=lr_callbacks,
#                     verbose=1)

In [17]:
# model_pk.save_weights(os.path.join(
#         weights_path, 'train_dense_model_pk_vdpm_mu_'+str(mu)+'.h5'))

In [18]:

# lr = 1e-5

# opt = Adam(lr)
# model_pk.compile(optimizer=opt, loss='mse')

# # lr_callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
# #                                                     factor=0.1,
# #                                                     patience=200,
# #                                                     verbose=0,
# #                                                     mode='auto',
# #                                                     min_delta=0.0001,
# #                                                     cooldown=0,
# #                                                     min_lr=1e-6)


In [19]:
# log_interval = 20
# lr_decay_factor = 0.8
# from tqdm.keras import TqdmCallback

# losses = []
# for i in range(100):
#     print('number of the outer loop:', i)

#     model_pk.get_layer('K_u_pred').trainable = False
#     model_pk.get_layer('DicNN').trainable = True
    
#     history_dic = model_pk.fit(x=[data_x, data_y, data_u],
#                     y=zeros_data_y_train,
#                     epochs=20,
#                     batch_size=200,
#                     callbacks=TqdmCallback(verbose=1),
#                     verbose=0)
    
#     model_pk.get_layer('K_u_pred').trainable = True
#     model_pk.get_layer('DicNN').trainable = False

#     history_K_u = model_pk.fit(x=[data_x, data_y, data_u],
#                     y=zeros_data_y_train,
#                     epochs=100,
#                     batch_size=200,
#                     callbacks=TqdmCallback(verbose=1),
#                     verbose=0)
    
#     if i % log_interval == 0:
#         losses.append(history_K_u.history['loss'][-1])

#         # Adjust learning rate:
#         if len(losses) > 2:
#             if losses[-1] > losses[-2]:
#                 print("Error increased. Decay learning rate")
#                 curr_lr = lr_decay_factor * model_pk.optimizer.lr
#                 model_pk.optimizer.lr = curr_lr

In [20]:

# model_pk.save_weights(os.path.join(
#     weights_path, 'test_model_pk_vdpm_mu_'+str(mu)+'.h5'))


In [21]:

# Linear Model: Dynamics is $Az +Bu$

dic_linear = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train)

solver_linear = KoopmanLinearDLSolver(
    dic=dic_linear, target_dim=target_dim, param_dim=param_dim, n_psi=n_psi)

model_linear = solver_linear.build_model()

solver_linear.build(model_linear,
                    data_x,
                    data_u,
                    data_y,
                    zeros_data_y_train,
                    epochs=linear_epochs,
                    batch_size=200,
                    lr=0.0001,
                    lr_patience=100,
                    lr_decay_factor=0.1,
                    lr_min=1e-6,
                    es_patience=50,
                    es_min_delta=1e-9,
                    filepath=os.path.join(
                        weights_path, 'model_linear_vdpm_mu_'+str(mu)+'.h5'))


100/100 [==============================] - 1s 3ms/step - loss: 0.4255 - val_loss: 0.3809 - lr: 1.0000e-04
number of the outer loop: 0
100/100 [==============================] - 0s 3ms/step - loss: 5.4526e-06 - val_loss: 5.5358e-06 - lr: 1.0000e-04
number of the outer loop: 1
100/100 [==============================] - 0s 3ms/step - loss: 1.8178e-07 - val_loss: 3.9593e-07 - lr: 1.0000e-04
number of the outer loop: 2
100/100 [==============================] - 0s 4ms/step - loss: 1.7876e-07 - val_loss: 3.9298e-07 - lr: 1.0000e-04
number of the outer loop: 3
100/100 [==============================] - 0s 3ms/step - loss: 1.7869e-07 - val_loss: 3.9343e-07 - lr: 1.0000e-04
number of the outer loop: 4
100/100 [==============================] - 0s 3ms/step - loss: 1.7869e-07 - val_loss: 3.9213e-07 - lr: 1.0000e-04
number of the outer loop: 5
100/100 [==============================] - 0s 3ms/step - loss: 1.7867e-07 - val_loss: 3.9244e-07 - lr: 1.0000e-04
number of the outer loop: 6
100/100 [=====

In [22]:

# Bilinear Model: Dynamics is $Az + \sum_{i=1}^{N_{u}}B_{i}zu_{i}$

dic_bilinear = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train)

solver_bilinear = KoopmanBilinearDLSolver(
    dic=dic_bilinear, target_dim=target_dim, param_dim=param_dim, n_psi=n_psi)

model_bilinear = solver_bilinear.build_model()

solver_bilinear.build(model_bilinear,
                        data_x,
                        data_u,
                        data_y,
                        zeros_data_y_train,
                        epochs=bilinear_epochs,
                        batch_size=200,
                        lr=0.0001,
                        lr_patience=100,
                        lr_decay_factor=0.1,
                        lr_min=1e-6,
                        es_patience=50,
                        es_min_delta=1e-9,
                        filepath=os.path.join(
                            weights_path, 'model_bilinear_vdpm_mu_'+str(mu)+'.h5'))


100/100 [==============================] - 1s 4ms/step - loss: 0.6025 - val_loss: 0.5811 - lr: 1.0000e-04
number of the outer loop: 0
100/100 [==============================] - 0s 3ms/step - loss: 2.8285e-05 - val_loss: 1.1201e-05 - lr: 1.0000e-04
number of the outer loop: 1
100/100 [==============================] - 0s 3ms/step - loss: 5.0311e-08 - val_loss: 8.1709e-08 - lr: 1.0000e-04
number of the outer loop: 2
100/100 [==============================] - 0s 3ms/step - loss: 3.0442e-08 - val_loss: 7.4473e-08 - lr: 1.0000e-04
number of the outer loop: 3
100/100 [==============================] - 0s 3ms/step - loss: 3.0398e-08 - val_loss: 7.4384e-08 - lr: 1.0000e-04
number of the outer loop: 4
100/100 [==============================] - 0s 3ms/step - loss: 3.0365e-08 - val_loss: 7.4388e-08 - lr: 1.0000e-04
number of the outer loop: 5
100/100 [==============================] - 0s 3ms/step - loss: 3.0316e-08 - val_loss: 7.4484e-08 - lr: 1.0000e-04
number of the outer loop: 6
100/100 [=====

In [26]:

# for mu, K_layer_size in zip(mu_list, K_layer_size_list):
#     load_data_and_train_models(mu, K_layer_size)
#     print('mu = ', mu, 'done')
#     print('K_layer_size = ', K_layer_size, 'done')

# load_data_and_train_models(mu=mu_list[0], K_layer_size=K_layer_size_list[0])
# print('mu = ', mu_list[0], 'done')
# print('K_layer_size = ', K_layer_size_list[0], 'done')

In [17]:

# def load_data_and_train_models(mu, K_layer_size):

#     # Load data
#     dict_data = np.load(os.path.join(
#         data_path, 'vdpm_data_mu_'+str(mu)+'.npy'), allow_pickle=True)

#     data_x = dict_data[()]['data_x']
#     data_y = dict_data[()]['data_y']
#     data_u = dict_data[()]['data_u']

#     # PK-NN
#     dic_pk = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train)
#     from koopmanlib.K_structure import Model_K_u_Layer, Model_K_u_Layer_One

#     model_K_u = Model_K_u_Layer_One(layer_sizes=K_layer_size, 
#                                     n_psi=n_psi,
#                                     activation='tanh')

#     solver_pk = KoopmanParametricDLSolver(
#         target_dim=target_dim, 
#         param_dim=param_dim, 
#         n_psi=n_psi, 
#         dic=dic_pk, 
#         model_K_u=model_K_u)

#     model_pk, model_K_u_pred_pk = solver_pk.generate_model()

#     model_pk.summary()

#     zeros_data_y_train = tf.zeros_like(dic_pk(data_y))

#     model_pk.compile(optimizer=Adam(0.001),
#                     loss='mse')
    
#     # model_pk.compile(optimizer=Adam(0.001),
#     #                 loss=my_loss)

#     lr_callbacks = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
#                                                         factor=0.1,
#                                                         patience=200,
#                                                         verbose=0,
#                                                         mode='auto',
#                                                         min_delta=0.0001,
#                                                         cooldown=0,
#                                                         min_lr=1e-6)

#     history = model_pk.fit(x=[data_x, data_y, data_u],
#                         y=zeros_data_y_train,
#                         epochs=pknn_epochs,
#                         batch_size=200,
#                         callbacks=lr_callbacks,
#                         verbose=1)


#     model_pk.save_weights(os.path.join(
#         weights_path, 'model_pk_vdpm_mu_'+str(mu)+'.h5'))

#     # # Linear Model: Dynamics is $Az +Bu$

#     # dic_linear = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train)

#     # solver_linear = KoopmanLinearDLSolver(
#     #     dic=dic_linear, target_dim=target_dim, param_dim=param_dim, n_psi=n_psi)

#     # model_linear = solver_linear.build_model()

#     # solver_linear.build(model_linear,
#     #                     data_x,
#     #                     data_u,
#     #                     data_y,
#     #                     zeros_data_y_train,
#     #                     epochs=linear_epochs,
#     #                     batch_size=200,
#     #                     lr=0.0001,
#     #                     log_interval=20,
#     #                     lr_decay_factor=0.1)

#     # model_linear.save_weights(os.path.join(
#     #     weights_path, 'model_linear_vdpm_mu_'+str(mu)+'.h5'))


#     # # Bilinear Model: Dynamics is $Az + \sum_{i=1}^{N_{u}}B_{i}zu_{i}$

#     # dic_bilinear = PsiNN(layer_sizes=dict_layer_size, n_psi_train=n_psi_train)

#     # solver_bilinear = KoopmanBilinearDLSolver(
#     #     dic=dic_bilinear, target_dim=target_dim, param_dim=param_dim, n_psi=n_psi)

#     # model_bilinear = solver_bilinear.build_model()

#     # solver_bilinear.build(model_bilinear,
#     #                       data_x,
#     #                       data_u,
#     #                       data_y,
#     #                       zeros_data_y_train,
#     #                       epochs=linear_epochs,
#     #                       batch_size=200,
#     #                       lr=0.0001,
#     #                       log_interval=20,
#     #                       lr_decay_factor=0.1)

#     # model_bilinear.save_weights(os.path.join(
#     #     weights_path, 'model_bilinear_vdpm_mu_'+str(mu)+'.h5'))

# # for mu, K_layer_size in zip(mu_list, K_layer_size_list):
# #     load_data_and_train_models(mu, K_layer_size)
# #     print('mu = ', mu, 'done')
# #     print('K_layer_size = ', K_layer_size, 'done')

# load_data_and_train_models(mu=mu_list[0], K_layer_size=K_layer_size_list[0])
# print('mu = ', mu_list[0], 'done')
# print('K_layer_size = ', K_layer_size_list[0], 'done')